In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

Tables and views in the database.

In [7]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


## Selecting album to purchase

The Chinook record store has just signed a deal with a new record label, and you've been tasked with selecting the first three albums that will be added to the store, from a list of four. All four albums are by artists that don't have any tracks in the store right now - we have the artist names, and the genre of music they produce:


| Artist Name        | Genre   |
|--------------------|---------|
|Regal               | Hip-Hop |
|Red Tone            | Punk    |
|Meteor and the Girls| Pop |
|Slim Jim Bites      |  Blues|

The record label specializes in artists from the USA, and they have given Chinook some money to advertise the new albums in the USA, so we're interested in finding out which genres sell the best in the USA.

In [22]:
%%sql
WITH usa_tracks AS (
    SELECT * FROM track t
    INNER JOIN invoice_line il ON il.track_id=t.track_id
    INNER JOIN invoice i ON i.invoice_id=il.invoice_id
    INNER JOIN customer c ON c.customer_id=i.customer_id
    WHERE c.country = 'USA'
)

SELECT 
    g.name genre, 
    COUNT(ut.track_id) tracks_sold_USA, 
    ROUND(CAST(COUNT(ut.track_id) AS FLOAT) / (SELECT COUNT(*) FROM usa_tracks)*100, 2) perc_sold_USA
FROM genre g
INNER JOIN usa_tracks ut ON ut.genre_id=g.genre_id
GROUP BY 1
ORDER BY 2 DESC;

Done.


genre,tracks_sold_USA,perc_sold_USA
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


The most sold genres in the USA are **Rock**, **Alternative & Punk**, **Metal** and **R&B/Soul**. In particular the **rock** is by far the most popular genre, with over 53% tracks sold in this genre. Based on this, we should buy more rock tracks, however the 4 artists we are proposed are not in this genre, therefore we should select artists from the next most popular genre: 
* Red Tone - Punk
* Slim Jim Bites - Blues
* Meteor and the Girls - Pop

## Analyzing Employee Sales Performance

In [43]:
%%sql

SELECT 
    e.first_name || " " || e.last_name employee,
    e.hire_date,
    ROUND(SUM(i.total),2) total_sales
FROM employee e
LEFT JOIN customer c ON e.employee_id=c.support_rep_id
LEFT JOIN invoice i ON c.customer_id=i.customer_id
GROUP BY 1
ORDER BY 3 DESC;

Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92
Andrew Adams,2016-08-14 00:00:00,None
Laura Callahan,2017-03-04 00:00:00,None
Michael Mitchell,2016-10-17 00:00:00,None
Nancy Edwards,2016-05-01 00:00:00,None
Robert King,2017-01-02 00:00:00,None


Out of 8 sales support agents, only 3 have customers and invoices associated with. Jane Peacock seems to have performed better than the others, she has the most sales. We notice that the amount of sales is proportional to the seniority level of the agent. However there are also more senior agents who have no sales, but this might be to several reasons (e.g. because they were not working in the period the data was collected).

## Analizing sales by country